# Demo Quantitative Stock Selection Model: Analyst Forecast Dispersion

Vamos 

The difficulty in generating this signal resides in merging data from two sources, the analyst forecasts and the prices. We will use **merge** function from pandas but, to do that, we need to generate a unique key for each row in both dataframes.

## Merge Two Data Frames by Keys

In [1]:
import pandas as pd
import numpy as np
df1 = pd.DataFrame({'key': ['A', 'B', 'C', 'D'],
                    'value1': [1, 2, 3, 5]})
df2 = pd.DataFrame({'key': ['E', 'D', 'C', 'B'],
                    'value2': [5, 6, 7, 8]})
display(df1)
display(df2)

,key,value1
0,A,1
1,B,2
2,C,3
3,D,5


,key,value2
0,E,5
1,D,6
2,C,7
3,B,8


Merge the two data frames on column "key". Use intersection of keys from both data frames. Preserve the order of keys from the left frame.

In [2]:
df3=pd.merge(df1,df2,on="key",how="inner")
df3

,key,value1,value2
0,B,2,8
1,C,3,7
2,D,5,6


Merge the two data frames on column "key". Use all keys from the left frame. Preserve the order of keys from the left frame.

In [3]:
#NaN will show up when the key is not available in the right frame
df3=pd.merge(df1,df2,on="key",how="left")
df3

,key,value1,value2
0,A,1,NaN
1,B,2,8.0
2,C,3,7.0
3,D,5,6.0


For more instruction about merge function, see https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html

## Analyst Forecast Dispersion 

$$
\frac{\text{STD of Forecast }{t}}{\text{Price at }t}
$$

## Step 1: Generate Keys to Merge Data Frames

In [4]:
df1 = pd.read_csv('classdata/IBES.csv')

In [5]:
#We only select the columns we need
df1=df1[["CUSIP","STATPERS","NUMEST","MEANEST","STDEV"]]
df1.head()

,CUSIP,STATPERS,NUMEST,MEANEST,STDEV
0,87482X10,20140417,4,0.52,0.03
1,87482X10,20140515,4,0.56,0.04
2,87482X10,20140619,4,0.56,0.04
3,87482X10,20140717,3,0.56,0.05
4,87482X10,20140814,5,1.18,0.06


Remove the records with no more than two analyst forecasts.

In [6]:
df1=df1[df1.NUMEST>2]

We generate the unique key for each row in df1 as "CUSIP"+"STATPERS", which represents the stock and the date from which this row is generated.

In [7]:
df1=df1[["CUSIP","STATPERS","STDEV"]]
df1["key"]=df1["CUSIP"]+df1["STATPERS"].astype(str)
df1.head()

,CUSIP,STATPERS,STDEV,key
0,87482X10,20140417,0.03,87482X1020140417
1,87482X10,20140515,0.04,87482X1020140515
2,87482X10,20140619,0.04,87482X1020140619
3,87482X10,20140717,0.05,87482X1020140717
4,87482X10,20140814,0.06,87482X1020140814


We then load the data file that contains the last closing price available to IBES before the statistics (STDEV) were calculated. 

In [8]:
df2 = pd.read_csv('classdata/ActualPrice.csv')
df2 = df2[["CUSIP","STATPERS","PRICE"]]
df2.head()

,CUSIP,STATPERS,PRICE
0,87482X10,20140417,13.75
1,87482X10,20140515,13.30
2,87482X10,20140619,14.40
3,87482X10,20140717,14.11
4,87482X10,20140814,13.76


Remove the records that have zero in prices.

In [9]:
df2=df2[df2.PRICE>0]

We generate the unique key for each row in df2 similarly.

In [10]:
df2["key"]=df2["CUSIP"]+df2["STATPERS"].astype(str)
df2=df2[["key","PRICE"]]
df2.head()

,key,PRICE
0,87482X1020140417,13.75
1,87482X1020140515,13.30
2,87482X1020140619,14.40
3,87482X1020140717,14.11
4,87482X1020140814,13.76


Merge the two data frames (df1 and df2) on column "key". Use intersection of keys from both data frames. Preserve the order of the left keys.

In [11]:
df3=pd.merge(df1,df2,on="key",how="inner")
df3.head()

,CUSIP,STATPERS,STDEV,key,PRICE
0,87482X10,20140417,0.03,87482X1020140417,13.75
1,87482X10,20140515,0.04,87482X1020140515,13.30
2,87482X10,20140619,0.04,87482X1020140619,14.40
3,87482X10,20140717,0.05,87482X1020140717,14.11
4,87482X10,20140814,0.06,87482X1020140814,13.76


## Step 2: Generate Signals and Available Quarters

Simply generate the signal in a new column.

In [12]:
df3["Signal"]=df3["STDEV"]/df3["PRICE"]
df3.head()

,CUSIP,STATPERS,STDEV,key,PRICE,Signal
0,87482X10,20140417,0.03,87482X1020140417,13.75,0.002182
1,87482X10,20140515,0.04,87482X1020140515,13.30,0.003008
2,87482X10,20140619,0.04,87482X1020140619,14.40,0.002778
3,87482X10,20140717,0.05,87482X1020140717,14.11,0.003544
4,87482X10,20140814,0.06,87482X1020140814,13.76,0.004360


In [13]:
#These libraries are loaded to move each date six months forwards (to the end of the returned month)
from dateutil.relativedelta import relativedelta

Generate **yyyymm** as $100\times year + 3\times quarter$ using list comprehension.

In [14]:
df3['STATPERS']=pd.to_datetime(df3['STATPERS'],format="%Y%m%d")
df3["year"]=[s.year for s in df3["STATPERS"]]
df3["quarter"]=[s.quarter for s in df3["STATPERS"]]
df3["Date"]=df3["year"]*100+df3["quarter"]*3
df3.head()

,CUSIP,STATPERS,STDEV,key,PRICE,Signal,year,quarter,Date
0,87482X10,2014-04-17,0.03,87482X1020140417,13.75,0.002182,2014,2,201406
1,87482X10,2014-05-15,0.04,87482X1020140515,13.30,0.003008,2014,2,201406
2,87482X10,2014-06-19,0.04,87482X1020140619,14.40,0.002778,2014,2,201406
3,87482X10,2014-07-17,0.05,87482X1020140717,14.11,0.003544,2014,3,201409
4,87482X10,2014-08-14,0.06,87482X1020140814,13.76,0.004360,2014,3,201409


Remove nan in df3 and sort df3.

In [15]:
df3.dropna(how='any',axis=0, inplace=True)
df3.sort_values(by=['CUSIP', 'Date'],inplace=True)
df3.reset_index(drop=True, inplace=True)

## Step 3: Generate Summary Statistics

For signals constructed using IBES data on analyst forecasts, the data are available at monthly frequency. We must only keep the signal from the last month of a quarter. For example, the standard deviation of analyst forecast is available in Jan, 
Feb, and Mar of 2012. To construct the signal of analyst forecast dispersion for 201203, you only need the standard deviation measured in March 2012.   

In [16]:
df3.groupby(["CUSIP"]).get_group("87482X10")

,CUSIP,STATPERS,STDEV,key,PRICE,Signal,year,quarter,Date
598096,87482X10,2014-04-17,0.03,87482X1020140417,13.75,0.002182,2014,2,201406
598097,87482X10,2014-05-15,0.04,87482X1020140515,13.30,0.003008,2014,2,201406
598098,87482X10,2014-06-19,0.04,87482X1020140619,14.40,0.002778,2014,2,201406
598099,87482X10,2014-07-17,0.05,87482X1020140717,14.11,0.003544,2014,3,201409
598100,87482X10,2014-08-14,0.06,87482X1020140814,13.76,0.004360,2014,3,201409
598101,87482X10,2014-09-18,0.06,87482X1020140918,14.76,0.004065,2014,3,201409
598102,87482X10,2014-10-16,0.06,87482X1020141016,13.61,0.004409,2014,4,201412
598103,87482X10,2014-11-20,0.03,87482X1020141120,13.95,0.002151,2014,4,201412
598104,87482X10,2014-12-18,0.03,87482X1020141218,14.17,0.002117,2014,4,201412
598105,87482X10,2015-01-15,0.03,87482X1020150115,13.17,0.002278,2015,1,201503


We use **groupby** to group the rows with the same "CUSIP" and "Date". Then we use **.tail(1)** to only keep the last row, which is the latest month in each quarter (the one we are supposed to use).

In [17]:
df3.groupby(["CUSIP","Date"]).get_group(("87482X10",201412))

,CUSIP,STATPERS,STDEV,key,PRICE,Signal,year,quarter,Date
598102,87482X10,2014-10-16,0.06,87482X1020141016,13.61,0.004409,2014,4,201412
598103,87482X10,2014-11-20,0.03,87482X1020141120,13.95,0.002151,2014,4,201412
598104,87482X10,2014-12-18,0.03,87482X1020141218,14.17,0.002117,2014,4,201412


In [18]:
df3.groupby(["CUSIP","Date"]).get_group(("87482X10",201412)).tail(1)

,CUSIP,STATPERS,STDEV,key,PRICE,Signal,year,quarter,Date
598104,87482X10,2014-12-18,0.03,87482X1020141218,14.17,0.002117,2014,4,201412


By removing **.get_group(("87482X10",201412))** in the code above, we can remove duplicated Dates in all groups.

In [19]:
df3=df3.groupby(["CUSIP","Date"]).tail(1)
df3.reset_index(drop=True, inplace=True)

Let's check if this works.

In [20]:
df3.groupby(["CUSIP"]).get_group("87482X10")

,CUSIP,STATPERS,STDEV,key,PRICE,Signal,year,quarter,Date
211237,87482X10,2014-06-19,0.04,87482X1020140619,14.40,0.002778,2014,2,201406
211238,87482X10,2014-09-18,0.06,87482X1020140918,14.76,0.004065,2014,3,201409
211239,87482X10,2014-12-18,0.03,87482X1020141218,14.17,0.002117,2014,4,201412
211240,87482X10,2015-03-19,0.02,87482X1020150319,14.93,0.001340,2015,1,201503
211241,87482X10,2015-06-18,0.05,87482X1020150618,16.66,0.003001,2015,2,201506
211242,87482X10,2015-09-17,0.03,87482X1020150917,16.38,0.001832,2015,3,201509
211243,87482X10,2015-12-17,0.01,87482X1020151217,17.66,0.000566,2015,4,201512
211244,87482X10,2016-03-17,0.04,87482X1020160317,17.31,0.002311,2016,1,201603
211245,87482X10,2016-04-14,0.04,87482X1020160414,18.53,0.002159,2016,2,201606


## Step 4: Map CUSIP to PERMNO

If we want to obtain the PERMNO corresponding to each stock, we can read "cusip.csv" where a mapping between CUSIP and PERMNO is provided. 

In [21]:
df4=pd.read_csv("classdata/cusip.csv")
df4["PERMNO"]=df4["PERMNO"].astype(str)
df4.head()

,PERMNO,NCUSIP
0,10000,68391610
1,10001,29269V10
2,10001,29274A10
3,10001,29274A20
4,10001,36720410


To do so, we just need to create a dictionary that maps NCUSIP to PERMNO. 

In [22]:
dictionary=dict(zip(df4.NCUSIP,df4.PERMNO))

In [23]:
df3["PERMNO"]=df3.CUSIP.map(dictionary)
df3[["PERMNO","CUSIP","Date","Signal"]].head()

,PERMNO,CUSIP,Date,Signal
0,NaN,00000000,201712,0.007539
1,NaN,00000000,201803,0.000558
2,NaN,00000117,200403,0.001815
3,NaN,00000117,200406,0.001808
4,NaN,00001216,201403,0.001451


In [24]:
df3.dropna(how='any',axis=0, inplace=True)
df3.reset_index(drop=True,inplace=True)
df3[["PERMNO","CUSIP","STATPERS","Date","Signal"]].head()

,PERMNO,CUSIP,STATPERS,Date,Signal
0,14945,00030710,2014-12-18,201412,0.001345
1,14945,00030710,2015-02-19,201503,0.001358
2,14945,00030710,2015-09-17,201509,0.001672
3,14945,00030710,2015-12-17,201512,0.000887
4,14945,00030710,2016-03-17,201603,0.001766


## Step 5: Generate Summary Statistics

In [25]:
df3[df3.quarter==4].groupby("Date")["Signal"].describe(percentiles=[0.1,0.9])

,count,mean,std,min,10%,50%,90%,max
Date,,,,,,,,
200012,3418.0,0.032777,0.493325,0.0,0.000307,0.002032,0.021728,19.500000
200112,2974.0,0.147527,2.902213,0.0,0.000321,0.001767,0.022554,139.000000
200212,2724.0,0.015942,0.146234,0.0,0.000339,0.001691,0.016098,3.428571
200312,2759.0,0.004325,0.030218,0.0,0.000285,0.001198,0.007515,1.402367
200412,2958.0,0.004370,0.038976,0.0,0.000283,0.001182,0.006927,1.966346
200512,3120.0,0.011268,0.251083,0.0,0.000313,0.001435,0.009190,13.000000
200612,3241.0,0.004757,0.018384,0.0,0.000348,0.001467,0.009124,0.709497
200712,3254.0,0.008877,0.063818,0.0,0.000388,0.001846,0.013768,2.862500
200812,2976.0,0.039510,0.333627,0.0,0.000773,0.004815,0.042463,14.000000


Save the columns we want to a csv file.

In [26]:
df3[["PERMNO","STATPERS","Date","Signal"]].to_csv("Signal2.csv",index=False)